In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# Initialize LLM
llm = ChatOpenAI(temperature=0.1)

# Examples for Few-Shot Learning
examples = [
    {"movie": "Titanic", "answer": "🚢❤️💔"},
    {"movie": "Jurassic Park", "answer": "🦖🌿🏃‍♂️"},
    {"movie": "The Matrix", "answer": "🕶️💊💻"},
]

# Few-Shot Template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=ChatPromptTemplate.from_messages([
        ("human", "{movie}"),
        ("ai", "{answer}")
    ])
)

# Final Prompt with Memory
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert. Respond with exactly three emojis."),
    MessagesPlaceholder(variable_name="chat_history"),  # Correctly formatted for memory
    few_shot_prompt,
    ("human", "{movie}")
])

# Memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True  # Ensures chat history is returned as a list of messages
)

# Function to Load Memory Properly
def load_memory(_):
    chat_history = memory.load_memory_variables({}).get("chat_history", [])
    return chat_history  # This must be a list

# LCEL Chain with Memory
chain = RunnablePassthrough().assign(chat_history=load_memory) | final_prompt | llm

# Function to Invoke Chain and Save Context
def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})  # Save interaction
    return result.content

# Example Runs
print(invoke_chain("Dune"))
print(invoke_chain("The Lion King"))

# Asking Memory what was the first movie asked about
print(invoke_chain("What was the first movie I asked about?"))



ValueError: variable chat_history should be a list of base messages, got {'chat_history': []}